In [1]:
import chromedriver_autoinstaller

In [3]:
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
from selenium.common.exceptions import *


In [89]:
def search_youtube_videos(keywords):
    url = f"https://www.youtube.com/results?search_query={(keywords)}"

    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(5)

    wait = WebDriverWait(driver, 10)
    cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
    cookie_consent.click()

    while True:
        scroll_height = 10
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(3)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
        if document_height_after == document_height_before:
            break

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    videos = soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer")

    rows = []
    for video in videos:
        name = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
        link = "https://www.youtube.com" + video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer")['href']
        info = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").get("aria-label").split(" by ")
        try:
            views = re.search(r'([\d,]+)\s*(views|view)', info[1]).group(1)
        except:
            views = 'unknown'

        try:
            date = re.search(r'(?P<count>\d+)\s+(?P<unit>\w+)\s+ago', info[1]).group()
        except:
            date = 'Unknown'

        try:
            duration =  re.search(r'\d+\s+(?:(?:minute|min|second|sec|hour)s?(?:,\s*)?)+\d*\s*(?:minute|second?)s?', info[1]).group()
        except:
            duration = 'Unknown'

        try:
            description = video.find("yt-formatted-string", class_="metadata-snippet-text style-scope ytd-video-renderer").text
        except:
            description = 'Unknown'

        rows.append([name, views, duration, date, description, link])

    df_yt = pd.DataFrame(rows, columns=['Name', 'views', 'duration', 'date', 'description', 'link'])
    
    driver.quit()

    return df_yt


In [122]:
def scrape_comments(df):
    comments = []

    for i in range(len(df)):
        url = df['link'][i]
        options = webdriver.ChromeOptions()
        options.add_argument("--incognito")
        options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(30)
        try:
            wait = WebDriverWait(driver, 10)
            cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
            cookie_consent.click()
        except TimeoutException:
            print('Cookie pop-up not loaded')

        time.sleep(30)
        
        driver.execute_script("window.scrollBy(0, 500);")

        try:
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.XPATH, '//ytd-comment-thread-renderer')))
            time.sleep(30)
            while True:
                scroll_height = 10
                document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
                driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
                time.sleep(2)
                document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
                if document_height_after == document_height_before:
                    break

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            for comment in soup.find_all('ytd-comment-thread-renderer'):
                text = comment.find('yt-formatted-string', {'class': 'style-scope ytd-comment-renderer'}).text
                likes = comment.find('span', {'id': 'vote-count-left'}).text.strip() if comment.find('span', {'id': 'vote-count-left'}) else 0
                replies = comment.find('button', {'class': 'yt-spec-button-shape-next yt-spec-button-shape-next--text yt-spec-button-shape-next--call-to-action yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--align-by-text'})
                if replies:
                    num_comments = replies.get('aria-label').split()[0]
                else:
                    num_comments = 0
                username_element = comment.find('a', {'id': 'author-text'}).find('span', {'class': 'style-scope ytd-comment-renderer'})
                if username_element:
                    username = username_element.text.replace('[','').replace(']','').replace('\n','')
                else:
                    username = ''
                link = url

                comments.append([username, text, likes, num_comments, link])
        except TimeoutException:
            comments.append(["ERROR", 'ERROR', 'ERROR', 'ERROR', url])
        finally:
            driver.quit()
            print(f'{url} done')

        df_comments = pd.DataFrame(comments, columns=['Username', 'Comment', 'Likes', 'Replies', 'link'])

    return df_comments


In [90]:
df_yt = search_youtube_videos('artificial intelligence')

In [91]:
df_yt

,Name,views,duration,date,description,link
0,What Is Artificial Intelligence? | 🔥Artificial...,"1,165,778","5 minutes, 28 seconds",3 years ago,"This video on ""What is Artificial Intelligence...",https://www.youtube.com/watch?v=ad79nYk2keg
1,How will AI change the world?,"727,683","5 minutes, 56 seconds",3 months ago,Explore the current limitations of artificial ...,https://www.youtube.com/watch?v=RzkD_rTEBYs
2,Artificial Intelligence: Last Week Tonight wit...,"5,213,390",Unknown,11 days ago,Artificial intelligence is increasingly becomi...,https://www.youtube.com/watch?v=Sqa8Zo2XWc4
3,"Midweek Marvels: Pete Buttigieg, Marianne Will...","16,277",Unknown,1 day ago,For entertainment purposes only. Just one guy'...,https://www.youtube.com/watch?v=H_xReIQ8DLo
4,The Danger of AI | Scary Technology | Artifici...,"74,296",Unknown,2 weeks ago,The Danger of AI - An examination of an altern...,https://www.youtube.com/watch?v=5guiUrgcblM
...,...,...,...,...,...,...
406,Top 10 AI Companies For 2023 | Top Artificial ...,"179,530","7 minutes, 32 seconds",3 months ago,"This video on ""Top 10 AI Companies For 2023"" b...",https://www.youtube.com/watch?v=l1__DHSAAKs
407,"Artificial Intelligence helped humans find 40,...","3,564",Unknown,7 months ago,Unknown,https://www.youtube.com/shorts/YJ-vA3dkVNg
408,Character Special | सीआईडी | CID | Artificial ...,"2,397,669",Unknown,10 months ago,Team CID has reached the location where people...,https://www.youtube.com/watch?v=XYJbasP66zw
409,How Machine Learning and Artificial Intelligen...,"1,304","4 minutes, 3 seconds",7 months ago,"In this 5 minutes video, we are going to expla...",https://www.youtube.com/watch?v=ZJNIbzk74J0


In [11]:
df_yt.to_csv('df_yt.csv', index = True)

In [27]:
df_scraped_comments = scrape_comments(df_yt)

NameError: name 'driver' is not defined

In [115]:
df_yt2  = df_yt.iloc[101:105].reset_index()

In [116]:
df_yt2 

,index,Name,views,duration,date,description,link
0,101,AI is Becoming Conscious! ELON MUSK was Right ...,"1,234,543","12 minutes, 11 seconds",3 months ago,AI i.e Artificial intelligence is getting more...,https://www.youtube.com/watch?v=R4iJMIgj6ZI
1,102,"Future of Artificial Intelligence (2030 - 10,0...","47,310","13 minutes, 21 seconds",6 months ago,___ Official Discord Server: https://discord.g...,https://www.youtube.com/watch?v=cwXnX49Bofk
2,103,"Artificial Intelligence: What, Why, How? | Abi...","107,859",Unknown,6 years ago,What does it mean for something to be Artifici...,https://www.youtube.com/watch?v=skEILdakHVc
3,104,In the Age of AI (full documentary) | FRONTLINE,"17,245,790","1 hour, 54 minutes",3 years ago,A documentary exploring how artificial intelli...,https://www.youtube.com/watch?v=5dZ_lvDgevk


In [88]:
def search_youtube_videos(keywords):
    url = f"https://www.youtube.com/results?search_query={(keywords)}"

    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(5)

    wait = WebDriverWait(driver, 10)
    cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
    cookie_consent.click()

    while True:
        scroll_height = 10
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(3)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
        if document_height_after == document_height_before:
            break

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    videos = soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer")

    rows = []
    for video in videos:
        name = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
        link = "https://www.youtube.com" + video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer")['href']
        info = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").get("aria-label").split(" by ")
        try:
            views = re.search(r'([\d,]+)\s*(views|view)', info[1]).group(1)
        except:
            views = 'unknown'

        try:
            date = re.search(r'(?P<count>\d+)\s+(?P<unit>\w+)\s+ago', info[1]).group()
        except:
            date = 'Unknown'

        try:
            duration =  re.search(r'\d+\s+(?:(?:minute|min|second|sec|hour)s?(?:,\s*)?)+\d*\s*(?:minute|second?)s?', info[1]).group()
        except:
            duration = 'Unknown'

        try:
            description = video.find("yt-formatted-string", class_="metadata-snippet-text style-scope ytd-video-renderer").text
        except:
            description = 'Unknown'

        rows.append([name, views, duration, date, description, link])

    df_yt = pd.DataFrame(rows, columns=['Name', 'views', 'duration', 'date', 'description', 'link'])
    
    driver.quit()

    return df_yt


In [53]:
df_scraped_comments = pd.concat([df_scraped_comments, df_scraped_comments2, df_scraped_comments3, df_scraped_comments4, df_scraped_comments5])

# reset index
df_scraped_comments = df_scraped_comments.reset_index(drop=True)

# export as CSV
df_scraped_comments.to_csv('df_scraped_comments.csv', index=False)

In [120]:
scrape_comments(df_yt2)

https://www.youtube.com/watch?v=R4iJMIgj6ZI done
https://www.youtube.com/watch?v=cwXnX49Bofk done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=skEILdakHVc done
https://www.youtube.com/watch?v=5dZ_lvDgevk done


,Username,Comment,Likes,Replies,link
0,Error: Comments could not be scraped,0,0,0,https://www.youtube.com/watch?v=R4iJMIgj6ZI
1,Error: Comments could not be scraped,0,0,0,https://www.youtube.com/watch?v=cwXnX49Bofk
2,Χρήστος Καπουράνης,"I could listen to hr for hours ,its not just t...",2,0,https://www.youtube.com/watch?v=skEILdakHVc
3,NickJRishwain,Abi is one of the smartest young women I know....,19,1,https://www.youtube.com/watch?v=skEILdakHVc
4,Wayne Biro,It is good to see a AI addressed enthusiastica...,1,0,https://www.youtube.com/watch?v=skEILdakHVc
...,...,...,...,...,...
1141,Teneisha Gordon,Outlook,0,0,https://www.youtube.com/watch?v=5dZ_lvDgevk
1142,Teneisha Gordon,Teneisha ehite,0,0,https://www.youtube.com/watch?v=5dZ_lvDgevk
1143,luke Smith,Can the current drivers of trucks be the “cont...,0,0,https://www.youtube.com/watch?v=5dZ_lvDgevk
1144,d loren,(Later that day at the bar)\n“Hahahhaha and di...,0,0,https://www.youtube.com/watch?v=5dZ_lvDgevk
